In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('final.csv')

In [3]:
X1 = df[['Sit_Time', 'Points_Count', 'Points_Percentage', 'Weight_Variance', 'Weight_of_cat', 'Stepping_In_Time', 'Times_Stepped', 'Event_start_time_hour']]

In [4]:
from sklearn.preprocessing import OneHotEncoder
from seaborn import load_dataset

ohe = OneHotEncoder()
transformed = ohe.fit_transform(X1[['Event_start_time_hour']])

X1[ohe.categories_[0]] = transformed.toarray()
X1 = X1.drop(columns = ['Event_start_time_hour'])
X1 = X1.drop(columns = [0])

C:\Users\kaushik.agrawal\Anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [5]:
X1_test = X1

In [6]:
X1

,Sit_Time,Points_Count,Points_Percentage,Weight_Variance,Weight_of_cat,Stepping_In_Time,Times_Stepped,1,2,3,...,14,15,16,17,18,19,20,21,22,23
0,44.274942,1772,0.291447,2.087515e+05,7357.0,0.025000,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,52.525000,2107,0.470313,1.586288e+05,6935.0,0.025000,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,43.900012,1760,0.429688,2.463771e+05,7357.0,1.374937,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.824935,448,0.168675,1.770790e+06,6983.0,0.025000,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,49.125000,1966,0.596481,1.221349e+05,7017.0,0.025000,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13070,48.558258,1947,0.822213,1.028918e+05,4116.0,0.025000,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
13071,18.190629,731,0.137613,1.259322e+06,4604.0,53.392769,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13072,77.868402,3128,0.565029,8.579901e+04,4416.0,0.025000,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13073,194.717593,7803,0.948808,3.478798e+04,4386.0,0.025000,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [7]:
import pickle
with open('elimination-classifier.pkl', 'rb') as f:
    elimination_classifier = pickle.load(f)
    
import pickle
with open('Urination-classifier.pkl', 'rb') as f:
    Urination_classifier = pickle.load(f)

In [8]:
y1 = elimination_classifier.predict(X1_test)
y1

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [9]:
y2 = Urination_classifier.predict(X1_test)
y2

array([1, 1, 1, ..., 1, 0, 1], dtype=int64)

In [10]:
Activity = []

for row in range(len(X1)):
    if y1[row]==0:
        Activity.append('Non-Elimination')
    if y1[row]==1:
        if y2[row]==1:
            Activity.append('Urination')
        else:
            Activity.append('Defecation')
            
df['Predicted Activity'] = Activity

In [11]:
result = df[['Device_ID', 'Event_ID', 'Event_start_time',
       'Name_of_cat', 'Weight_of_cat', 'Predicted Activity']]
result.to_csv('result.csv')